# Autor: Jesús Galán Llano
## Correo: jgalan279@alumno.uned.es

Crear el entorno de trabajo.

In [1]:
! mkdir -p practica1

In [2]:
import os

os.chdir("/media/notebooks/practicas/practica1")

In [3]:
! pwd

/media/notebooks/practicas/practica1


# Ejercicio 1. MapReduce

Para la realización de este ejercicio se ha partido desde el archivo outputlocal que contiene la lista de países y clientes ya unificados.

In [4]:
cat outputlocal

"PR"	"Puerto Rico,regular"
"PT"	"Portugal,bueno"
"QA"	"Qatar,bueno"
"US"	"United States,malo"
"US"	"United States,regular"
"US"	"United States,malo"
"US"	"United States,bueno"
"ML"	"Mali,malo"
"GB"	"United Kingdom,regular"
"GB"	"United Kingdom,malo"
"GB"	"United Kingdom,malo"
"GB"	"United Kingdom,malo"
"GG"	"Guernsey,malo"
"RE"	"R\u00e9union,regular"
"RO"	"Romania,malo"
"SB"	"Solomon Islands,regular"
"SI"	"Slovenia,regular"
"MT"	"Malta,regular"
"MV"	"Maldives,malo"
"IT"	"Italy,malo"
"IT"	"Italy,regular"
"AE"	"United Arab Emirates,regular"
"LK"	"Sri Lanka,malo"
"LK"	"Sri Lanka,malo"
"TM"	"Turkmenistan,regular"
"TR"	"Turkey,bueno"
"TT"	"Trinidad and Tobago,regular"
"ES"	"Spain,regular"
"ES"	"Spain,bueno"
"ES"	"Spain,regular"
"ES"	"Spain,malo"
"ES"	"Spain,bueno"
"ES"	"Spain,malo"
"ES"	"Spain,bueno"
"GN"	"Guinea,bueno"
"GS"	"South Georgia and the South Sandwich Islands,malo"
"GS"	"South Georgia and the South Sandwich Islands,bueno"
"GT"	"Guatemala,regular"
"GU"	"Guam,bueno"
"GU"	"Guam,buen

Este archivo se ha obtenido tras ejecutar el programa mrjob-join, que venía a modo de ejemplo al comienzo de la práctica. Este código se ha modificado únicamente el formato de salida de la función reduce para simplificar la lectura de los datos en los siguientes apartados. El código se incluye a continuación.

In [5]:
%%writefile mrjob-ejercicio.py
import sys, os, re
from mrjob.job import MRJob

class MRJoin(MRJob):

  # Realiza la ordenacion secundaria
  SORT_VALUES = True

  def mapper(self, _, line):
    splits = line.rstrip("\n").split(",")

    if len(splits) == 2: # datos de paises
      symbol = 'A' # ordenamos los paises antes que los datos de personas
      country2digit = splits[1]
      yield country2digit, [symbol, splits]
    else: #  datos de personas
      symbol = 'B'
      country2digit = splits[2]
      yield country2digit, [symbol, splits]
    
    line = line.rstrip("\n").split('	')
    country, value = line[1].split(',')
    country, value = country.replace('"', ''), value.replace('"', '')
    if value == 'bueno':
        yield country, 1

  def reducer(self, key, values):
    countries = [] # paises primero ya que llevan la clave 'A'
    for value in values:
      if value[0] == 'A':
        countries.append(value)
      if value[0] == 'B':
        for country in countries:
            # Modificar el formato de salida original para facilitar el procesamiento
            yield key, f'{country[1][0]},{value[1][1]}'

if __name__ == '__main__':
    MRJoin.run()

Overwriting mrjob-ejercicio.py


### Ejercicio 1.1: Contador de clientes valorados por países.

Este programa cuenta el número de clientes "buenos" por cada país. El número de pasos del programa es de dos: en primer lugar se realiza una función map y seguidamente se ejecuta una función reduce. La función map se encarga de leer los datos del archivo de entrada y de identificar a los clientes categorizados como buenos. Para ello, divide los datos de entrada, separados por el caracter ",", y guarda el país y la categoría. Si la categoría tiene el valor "bueno" devuelve un par <clave,valor> donde la clave es el nombre del país y el valor es 1. En el último paso, la función reduce encarga de calcular el sumatorio de la entrada para todos los países. Para este cálculo se utiliza la función sum() de Python. 

In [6]:
%%writefile ejercicio11.py
import sys, os, re
from mrjob.job import MRJob

class MRJoin(MRJob):

  # Realiza la ordenacion secundaria
  SORT_VALUES = True

  def mapper(self, key, line):
    line = line.rstrip("\n").split('	')
    country, value = line[1].split(',')
    country, value = country.replace('"', ''), value.replace('"', '')
    if value == 'bueno':
        yield country, 1
    
  def reducer(self, key, values):
    yield (key, sum(values)) 
        

if __name__ == '__main__':
    MRJoin.run()

Overwriting ejercicio11.py


Primero ejecutamos el códigoen local y luego en Hadoop

In [7]:
! python3 ejercicio11.py outputlocal > outputlocal11

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/ejercicio11.root.20211117.234225.913325
Running step 1 of 1...
job output is in /tmp/ejercicio11.root.20211117.234225.913325/output
Streaming final output from /tmp/ejercicio11.root.20211117.234225.913325/output...
Removing temp directory /tmp/ejercicio11.root.20211117.234225.913325...


In [8]:
! cat outputlocal11

"South Georgia and the South Sandwich Islands"	1
"Spain"	3
"Qatar"	1
"Somalia"	1
"Canada"	1
"Guam"	3
"Turkey"	1
"United States"	1
"South Africa"	1
"South Sudan"	1
"Guinea"	1
"Portugal"	1


In [9]:
! hdfs dfs -mkdir /tmp/ej11
! hdfs dfs -put /media/notebooks/practicas/practica1/outputlocal /tmp/ej11

In [10]:
! hdfs dfs -ls  /tmp/ej11

Found 1 items
-rw-r--r--   3 root supergroup       1236 2021-11-18 00:42 /tmp/ej11/outputlocal


In [11]:
! hdfs dfs -rm -r /tmp/ej11-output/*
! hdfs dfs -rmdir /tmp/ej11-output

rm: `/tmp/ej11-output/*': No such file or directory
rmdir: `/tmp/ej11-output': No such file or directory


In [12]:
! python3 ejercicio11.py hdfs:///tmp/ej11/* -r hadoop --output-dir hdfs:///tmp/ej11-output

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /app/hadoop-3.3.1/bin...
Found hadoop binary: /app/hadoop-3.3.1/bin/hadoop
Using Hadoop version 3.3.1
Looking for Hadoop streaming jar in /app/hadoop-3.3.1...
Found Hadoop streaming jar: /app/hadoop-3.3.1/share/hadoop/tools/lib/hadoop-streaming-3.3.1.jar
Creating temp directory /tmp/ejercicio11.root.20211117.234243.810072
uploading working dir files to hdfs:///user/root/tmp/mrjob/ejercicio11.root.20211117.234243.810072/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/ejercicio11.root.20211117.234243.810072/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar4457515230502276056/] [] /tmp/streamjob3386770937057026826.jar tmpDir=null
  Connecting to ResourceManager at yarnmaster/172.18.0.7:8032
  Connecting to ResourceManager at yarnmaster/172.18.0.7:8032
  Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_163

In [13]:
! hdfs dfs -tail /tmp/ej11-output/part-00000

"Canada"	1
"Guam"	3
"Guinea"	1
"Portugal"	1
"Qatar"	1
"Somalia"	1
"South Africa"	1
"South Georgia and the South Sandwich Islands"	1
"South Sudan"	1
"Spain"	3
"Turkey"	1
"United States"	1


### Ejercicio 1.2. País con mejores clientes. Mostrar el país con mayor número de clientes valorados como "bueno".

Este programa permite obtener el país con mejores clientes considerados "buenos". El programa necesita de cuatro pasos totales, si se cuentan los dos primeros pasos que se corresponden con las funciones map y reduce del programa del apartado 1.1. La nueva función mapper, mapper2, recibe de entrada la salida del programa del apartado 1.1, es decir, un conjunto de <claves,valor> donde la clave es el nombre del país y el valor es el número de clientes categorizados como buenos. Una vez recibe los datos, la función realiza una conversión de los valores a entero, por si hubiera algún dato que no fuera correcto, y retorna todos los pares <clave,valor>, pero sin devolver la clave. De esta forma se envía un generador a la siguiente función reduce para que pueda utilizar todos los datos. Por último, la función reduce se encarga de realizar una operación de maximización sobre los datos recibidos para calcular el país con mayor número de clientes considerados buenos. Utiliza la función max() de Python para realizar este cálculo. Para establecer un orden de ejecución de las funciones map y reduce utilizamos el método steps.

In [14]:
%%writefile ejercicio12.py
import sys, os, re
from mrjob.job import MRJob
from mrjob.step import MRStep

class MRJoin(MRJob):

  # Realiza la ordenacion secundaria
  SORT_VALUES = True

  def mapper(self, key, line):
    line = line.rstrip("\n").split('	')
    country, value = line[1].split(',')
    country, value = country.replace('"', ''), value.replace('"', '')
    if value == 'bueno':
        yield country, 1
    
  def reducer(self, key, values):
    yield (key, sum(values))
    
  def mapper2(self, key, values):
    values = int(values)
    yield None, (values,key)
    
  def reducer2(self, _, values):
    yield (max(values))
    
  def steps(self):
    return [
        MRStep(mapper=self.mapper, reducer=self.reducer),
        MRStep(mapper=self.mapper2, reducer=self.reducer2)
    ]
         

if __name__ == '__main__':
    MRJoin.run()

Overwriting ejercicio12.py


Primero ejecutamos el códigoen local y luego en Hadoop

In [4]:
! python3 ejercicio12.py outputlocal > outputlocal12

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/ejercicio12.root.20211117.231108.125077
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/ejercicio12.root.20211117.231108.125077/output
Streaming final output from /tmp/ejercicio12.root.20211117.231108.125077/output...
Removing temp directory /tmp/ejercicio12.root.20211117.231108.125077...


In [5]:
! cat outputlocal12

3	"Spain"


In [7]:
! hdfs dfs -mkdir /tmp/ej12
! hdfs dfs -put /media/notebooks/practicas/practica1/outputlocal  /tmp/ej12

mkdir: `/tmp/ej12': File exists
put: `/tmp/ej12/outputlocal': File exists


In [8]:
! hdfs dfs -ls  /tmp/ej12

Found 1 items
-rw-r--r--   3 root supergroup       1236 2021-11-18 00:11 /tmp/ej12/outputlocal


In [9]:
! hdfs dfs -rm -r /tmp/ej12-output/*
! hdfs dfs -rmdir /tmp/ej12-output

rm: `/tmp/ej12-output/*': No such file or directory
rmdir: `/tmp/ej12-output': No such file or directory


In [10]:
! python3 ejercicio12.py hdfs:///tmp/ej12/* -r hadoop --output-dir hdfs:///tmp/ej12-output

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /app/hadoop-3.3.1/bin...
Found hadoop binary: /app/hadoop-3.3.1/bin/hadoop
Using Hadoop version 3.3.1
Looking for Hadoop streaming jar in /app/hadoop-3.3.1...
Found Hadoop streaming jar: /app/hadoop-3.3.1/share/hadoop/tools/lib/hadoop-streaming-3.3.1.jar
Creating temp directory /tmp/ejercicio12.root.20211117.231153.727142
uploading working dir files to hdfs:///user/root/tmp/mrjob/ejercicio12.root.20211117.231153.727142/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/ejercicio12.root.20211117.231153.727142/files/
Running step 1 of 2...
  packageJobJar: [/tmp/hadoop-unjar3507074732095050767/] [] /tmp/streamjob8335111148641592053.jar tmpDir=null
  Connecting to ResourceManager at yarnmaster/172.18.0.2:8032
  Connecting to ResourceManager at yarnmaster/172.18.0.2:8032
  Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_163

In [12]:
! hdfs dfs -tail /tmp/ej12-output/part-00000

3	"Spain"


### Ejercicio 1.3. País con mejores clientes mejorado

Este programa resuelve el problema del apartado 1.2, que es devolver un único país en el caso de que haya más de uno que tenga el mismo número máximo de clientes considerados "buenos". El programa consta de las mismas fases que el programa del apartado anterior y todas realizan la misma funcionalidad, a excepción de la segunda función reduce. Esta función se encarga primero de guardar los datos de entrada como una lista y de calcular su valor máximo, que se corresponde con el valor más alto de clientes categorizados como "buenos". Tras obtener este máximo, la función recorre la lista de países y, en aquellos donde el máximo coincida con el número de clientes "buenos", devuelve una tupla <clave,valor> con el nombre del país. De esta forma se obtiene una serie de países y no únicamente uno.

In [13]:
%%writefile ejercicio13.py
import sys, os, re
from mrjob.job import MRJob
from mrjob.step import MRStep

class MRJoin(MRJob):

  # Realiza la ordenacion secundaria
  SORT_VALUES = True

  def mapper(self, key, line):
    line = line.rstrip("\n").split('	')
    country, value = line[1].split(',')
    country, value = country.replace('"', ''), value.replace('"', '')
    if value == 'bueno':
        yield country, 1
        
  def reducer(self, key, values):
    yield (key, sum(values))
    
  def mapper2(self, key, values):
    yield None, (values,key)
    
  def reducer2(self, key, values):
    values = list(values)
    maxv = (max(values))
    for value in values:
        if value[0] == maxv[0]:
            yield value[0], value[1]    

  def steps(self):
    return [
        MRStep(mapper=self.mapper, reducer=self.reducer),
        MRStep(mapper=self.mapper2, reducer=self.reducer2)
    ]
         

if __name__ == '__main__':
    MRJoin.run()

Overwriting ejercicio13.py


In [14]:
! python3 ejercicio13.py outputlocal > outputlocal13

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/ejercicio13.root.20211117.231347.061460
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/ejercicio13.root.20211117.231347.061460/output
Streaming final output from /tmp/ejercicio13.root.20211117.231347.061460/output...
Removing temp directory /tmp/ejercicio13.root.20211117.231347.061460...


Primero ejecutamos el códigoen local y luego en Hadoop

In [15]:
! python3 ejercicio13.py outputlocal > outputlocal13

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/ejercicio13.root.20211117.231349.302033
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/ejercicio13.root.20211117.231349.302033/output
Streaming final output from /tmp/ejercicio13.root.20211117.231349.302033/output...
Removing temp directory /tmp/ejercicio13.root.20211117.231349.302033...


In [16]:
! cat outputlocal13

3	"Guam"
3	"Spain"


In [17]:
! hdfs dfs -mkdir /tmp/ej13
! hdfs dfs -put /media/notebooks/practicas/practica1/outputlocal  /tmp/ej13

In [18]:
! hdfs dfs -ls  /tmp/ej13

Found 1 items
-rw-r--r--   3 root supergroup       1236 2021-11-18 00:14 /tmp/ej13/outputlocal


In [19]:
! hdfs dfs -rm -r /tmp/ej13-output/*
! hdfs dfs -rmdir /tmp/ej13-output

rm: `/tmp/ej13-output/*': No such file or directory
rmdir: `/tmp/ej13-output': No such file or directory


In [20]:
! python3 ejercicio13.py hdfs:///tmp/ej13/* -r hadoop --output-dir hdfs:///tmp/ej13-output

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /app/hadoop-3.3.1/bin...
Found hadoop binary: /app/hadoop-3.3.1/bin/hadoop
Using Hadoop version 3.3.1
Looking for Hadoop streaming jar in /app/hadoop-3.3.1...
Found Hadoop streaming jar: /app/hadoop-3.3.1/share/hadoop/tools/lib/hadoop-streaming-3.3.1.jar
Creating temp directory /tmp/ejercicio13.root.20211117.231443.929168
uploading working dir files to hdfs:///user/root/tmp/mrjob/ejercicio13.root.20211117.231443.929168/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/ejercicio13.root.20211117.231443.929168/files/
Running step 1 of 2...
  packageJobJar: [/tmp/hadoop-unjar5206616686777112180/] [] /tmp/streamjob4518172108230057195.jar tmpDir=null
  Connecting to ResourceManager at yarnmaster/172.18.0.2:8032
  Connecting to ResourceManager at yarnmaster/172.18.0.2:8032
  Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_163

In [21]:
! hdfs dfs -tail /tmp/ej13-output/part-00000

3	"Guam"
3	"Spain"


# Ejercicio 2. Hive

En primer lugar, se comprueba la conexión con la base de datos

In [23]:
!beeline -u "jdbc:hive2://localhost:10000/" -e "SHOW DATABASES;"

Connecting to jdbc:hive2://localhost:10000/
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118004629_44ba1bb7-834c-44c9-bcb0-0cc8eb21136a): SHOW DATABASES
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:database_name, type:string, comment:from deserializer)], properties:null)
INFO  : Completed compiling command(queryId=root_20211118004629_44ba1bb7-834c-44c9-bcb0-0cc8eb21136a); Time taken: 0.011 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118004629_44ba1bb7-834c-44c9-bcb0-0cc8eb21136a): SHOW DATABASES
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20211118004629_44ba1bb7-834c-44c9-bcb0-0cc8eb21136a); T

In [24]:
!beeline -u "jdbc:hive2://localhost:10000/" -n "bigdata" -p "bigdata" -e "SHOW DATABASES;"

Connecting to jdbc:hive2://localhost:10000/
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118004635_0a1c8fce-4671-42a4-9942-923baeae717d): SHOW DATABASES
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:database_name, type:string, comment:from deserializer)], properties:null)
INFO  : Completed compiling command(queryId=root_20211118004635_0a1c8fce-4671-42a4-9942-923baeae717d); Time taken: 0.013 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118004635_0a1c8fce-4671-42a4-9942-923baeae717d): SHOW DATABASES
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20211118004635_0a1c8fce-4671-42a4-9942-923baeae717d); T

## 2.1. Crear las tablas necesarias para almacenar datos

Se ha decidido crear las tablas de forma interna debido a: facilitar la manipulación de los datos a Hive y porque en el contexto de la práctica ninguna otra aplicación va a hacer a los datos que se almacenen. En el caso de que hubiera otras aplicaciones que quisieran acceder a los datos se hubiera adoptado por crear las tablas de forma externa. 

En primer lugar, creamos la base de datos que almacenará las tablas que serán creadas

In [25]:
sql_statement = "CREATE DATABASE IF NOT EXISTS practica \
COMMENT 'practica1' \
WITH DBPROPERTIES ('Creada por' = 'BigData', 'Fecha' = '09/11/21')"

!beeline -u "jdbc:hive2://localhost:10000/"  -e "{sql_statement}"

Connecting to jdbc:hive2://localhost:10000/
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118004641_fdf4f16e-e2f2-487c-a776-e928fbdbf1d2): CREATE DATABASE IF NOT EXISTS practica COMMENT 'practica1' WITH DBPROPERTIES ('Creada por' = 'BigData', 'Fecha' = '09/11/21')
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20211118004641_fdf4f16e-e2f2-487c-a776-e928fbdbf1d2); Time taken: 0.016 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118004641_fdf4f16e-e2f2-487c-a776-e928fbdbf1d2): CREATE DATABASE IF NOT EXISTS practica COMMENT 'practica1' WITH DBPROPERTIES ('Creada por' = 'BigData', 'Fecha' = '09/11/21')
IN

Comprobamos que se ha creado la base de datos correctamente

In [26]:
!beeline -u "jdbc:hive2://localhost:10000/" -e "SHOW DATABASES;"

Connecting to jdbc:hive2://localhost:10000/
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118004647_1207e649-fd8f-454d-9273-7f17c95e9b3d): SHOW DATABASES
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:database_name, type:string, comment:from deserializer)], properties:null)
INFO  : Completed compiling command(queryId=root_20211118004647_1207e649-fd8f-454d-9273-7f17c95e9b3d); Time taken: 0.012 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118004647_1207e649-fd8f-454d-9273-7f17c95e9b3d): SHOW DATABASES
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20211118004647_1207e649-fd8f-454d-9273-7f17c95e9b3d); T

Creamos las tablas en la base de datos

In [27]:
sql_statement = "DROP TABLE author;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement};"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118004652_a36daab2-b9c9-4267-a90c-ef15da5f1942): DROP TABLE author
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20211118004652_a36daab2-b9c9-4267-a90c-ef15da5f1942); Time taken: 0.016 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118004652_a36daab2-b9c9-4267-a90c-ef15da5f1942): DROP TABLE author
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20211118004652_a36daab2-b9c9-4267-a90c-ef15da5f1942); Time taken: 0.01 seconds
INFO  : OK
INFO  : Concurrency 

In [28]:
sql_statement = "CREATE TABLE IF NOT EXISTS author(author_id INT, author_name STRING) \
COMMENT 'Autores con id y nombre' \
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\;' \
TBLPROPERTIES ('Autor' = 'Bigdata', 'Fecha' = '09/11/21', 'skip.header.line.count'='1');"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "USE practica; {sql_statement}"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118004657_febd442a-a3ba-4105-a0d0-eccebfa6ecef): USE practica
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20211118004657_febd442a-a3ba-4105-a0d0-eccebfa6ecef); Time taken: 0.015 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118004657_febd442a-a3ba-4105-a0d0-eccebfa6ecef): USE practica
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20211118004657_febd442a-a3ba-4105-a0d0-eccebfa6ecef); Time taken: 0.011 seconds
INFO  : OK
INFO  : Concurrency mode is d

In [29]:
sql_statement = "DROP TABLE dataset;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement};"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118004703_70bbf1d3-238c-4c6b-ad9b-e7bbb6b6a459): DROP TABLE dataset
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20211118004703_70bbf1d3-238c-4c6b-ad9b-e7bbb6b6a459); Time taken: 0.014 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118004703_70bbf1d3-238c-4c6b-ad9b-e7bbb6b6a459): DROP TABLE dataset
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20211118004703_70bbf1d3-238c-4c6b-ad9b-e7bbb6b6a459); Time taken: 0.01 seconds
INFO  : OK
INFO  : Concurrenc

In [30]:
sql_statement = "CREATE TABLE IF NOT EXISTS dataset(title STRING, author_id INT, bestsellers_rank INT, \
imprint STRING, publication_date DATE, rating_avg FLOAT, rating_count INT) \
COMMENT 'Libros con su informacion y autores' \
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\;' \
TBLPROPERTIES ('Autor' = 'Bigdata', 'Fecha' = '09/11/21', 'skip.header.line.count'='1');"

!beeline -u "jdbc:hive2://localhost:10000/" -e "USE practica; {sql_statement}"

Connecting to jdbc:hive2://localhost:10000/
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118004708_c2adec38-b4a5-4ebb-93e0-a4d489a79b09): USE practica
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20211118004708_c2adec38-b4a5-4ebb-93e0-a4d489a79b09); Time taken: 0.015 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118004708_c2adec38-b4a5-4ebb-93e0-a4d489a79b09): USE practica
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20211118004708_c2adec38-b4a5-4ebb-93e0-a4d489a79b09); Time taken: 0.02 seconds
INFO  : OK
INFO  : Concurrency mode is disabled, 

Comprobamos que ambas tablas se han creado correctamente

In [31]:
!beeline -u "jdbc:hive2://localhost:10000/practica" -e "SHOW TABLES;"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118004714_e1406f9c-32f6-4c03-956f-f769effdee2c): SHOW TABLES
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:tab_name, type:string, comment:from deserializer)], properties:null)
INFO  : Completed compiling command(queryId=root_20211118004714_e1406f9c-32f6-4c03-956f-f769effdee2c); Time taken: 0.016 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118004714_e1406f9c-32f6-4c03-956f-f769effdee2c): SHOW TABLES
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20211118004714_e1406f9c-32f6-4c03-956f-f769effdee2c); Time

### 2.2. Importar los datos en las tablas creadas

Primero, copiamos los datasets en HDFS

In [32]:
!hadoop fs -copyFromLocal authors.csv /user/root/

In [33]:
!hadoop fs -copyFromLocal dataset.csv /user/root/

Una vez cargados los datasets en HDFS, los importamos en su tabla correspondiente comprobando que se han cargado correctamente

In [34]:
sql_statement = "LOAD DATA INPATH 'authors.csv' INTO TABLE author;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement};"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118004727_d2cb26fb-fd93-4c21-ac05-1981cd6d118f): LOAD DATA INPATH 'authors.csv' INTO TABLE author
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20211118004727_d2cb26fb-fd93-4c21-ac05-1981cd6d118f); Time taken: 0.216 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118004727_d2cb26fb-fd93-4c21-ac05-1981cd6d118f): LOAD DATA INPATH 'authors.csv' INTO TABLE author
INFO  : Starting task [Stage-0:MOVE] in serial mode
INFO  : Loading data to table practica.author from hdfs://namenode:8020/user/root/authors.csv
IN

Comprobamos que los datos están cargados

In [35]:
sql_statement = "SELECT * FROM author LIMIT 10;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement}"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118004735_bd4610a6-d63a-4963-8d87-ce089869f7c8): SELECT * FROM author LIMIT 10
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:author.author_id, type:int, comment:null), FieldSchema(name:author.author_name, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20211118004735_bd4610a6-d63a-4963-8d87-ce089869f7c8); Time taken: 0.973 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118004735_bd4610a6-d63a-4963-8d87-ce089869f7c8): SELECT * FROM author LIMIT 10
INFO  : Completed executing command(queryId=root_20211118004735_b

In [ ]:
# No ejecutar
sql_statement = "LOAD DATA INPATH 'dataset.csv' INTO TABLE dataset;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement};"

Debido a que el formato de la fecha del archivo dataset.csv no se se corresponde con el que soporta HIVE se debe 
realizar un proceso intermedio para su carga. 
Este proceso consiste en crear una tabla temporal que almacene la fecha como string para seguidamente
cargar todos los datos de esa tabla en otra. El formato de la fecha se modifica antes de la carga.

In [36]:
sql_statement = "DROP TABLE dataset_inter;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement};"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118004742_a485cb7d-eff0-4034-8abc-681145865814): DROP TABLE dataset_inter
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20211118004742_a485cb7d-eff0-4034-8abc-681145865814); Time taken: 0.015 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118004742_a485cb7d-eff0-4034-8abc-681145865814): DROP TABLE dataset_inter
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20211118004742_a485cb7d-eff0-4034-8abc-681145865814); Time taken: 0.014 seconds
INFO  : OK
INFO 

In [37]:
sql_statement = "CREATE TABLE IF NOT EXISTS dataset_inter(title STRING, author_id INT, bestsellers_rank INT, \
imprint STRING, publication_date STRING, rating_avg FLOAT, rating_count INT) \
COMMENT 'Libros con su informacion y autores' \
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\;' \
TBLPROPERTIES ('Autor' = 'Bigdata', 'Fecha' = '09/11/21', 'skip.header.line.count'='1');"

!beeline -u "jdbc:hive2://localhost:10000/" -e "USE practica; {sql_statement}"

Connecting to jdbc:hive2://localhost:10000/
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118004747_bd069a6e-28c3-422d-846d-b730b00d8c19): USE practica
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20211118004747_bd069a6e-28c3-422d-846d-b730b00d8c19); Time taken: 0.014 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118004747_bd069a6e-28c3-422d-846d-b730b00d8c19): USE practica
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20211118004747_bd069a6e-28c3-422d-846d-b730b00d8c19); Time taken: 0.009 seconds
INFO  : OK
INFO  : Concurrency mode is disabled,

Cargamos los datos en la tabla intermedia

In [38]:
!hadoop fs -copyFromLocal dataset.csv /user/root/

copyFromLocal: `/user/root/dataset.csv': File exists


In [39]:
sql_statement = "LOAD DATA INPATH 'dataset.csv' INTO TABLE dataset_inter;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement};"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118004755_8b13aa61-58d6-4e10-933c-b17f6dd32983): LOAD DATA INPATH 'dataset.csv' INTO TABLE dataset_inter
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20211118004755_8b13aa61-58d6-4e10-933c-b17f6dd32983); Time taken: 0.034 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118004755_8b13aa61-58d6-4e10-933c-b17f6dd32983): LOAD DATA INPATH 'dataset.csv' INTO TABLE dataset_inter
INFO  : Starting task [Stage-0:MOVE] in serial mode
INFO  : Loading data to table practica.dataset_inter from hdfs://namenode:8020/use

Comprobamos que los datos están cargados

In [40]:
sql_statement = "SELECT * FROM dataset_inter LIMIT 5;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement}"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118004802_056d7afd-8355-44fc-b33b-a99b44da1aea): SELECT * FROM dataset_inter LIMIT 5
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:dataset_inter.title, type:string, comment:null), FieldSchema(name:dataset_inter.author_id, type:int, comment:null), FieldSchema(name:dataset_inter.bestsellers_rank, type:int, comment:null), FieldSchema(name:dataset_inter.imprint, type:string, comment:null), FieldSchema(name:dataset_inter.publication_date, type:string, comment:null), FieldSchema(name:dataset_inter.rating_avg, type:float, comment:null), FieldSchema(name:dataset_inter.rating_count, type:int, comment:null)], properties:null)
INFO  : Co

Cargamos los datos en la tabla definitiva partiendo de la tabla intermedia y modificando el formato de la fecha

In [41]:
sql_statement = " INSERT INTO TABLE dataset \
SELECT title, author_id, bestsellers_rank, imprint, cast(to_date(from_unixtime(unix_timestamp(publication_date, 'dd/MM/yyyy'))) as date) as publication_date, \
rating_avg, rating_count \
FROM dataset_inter;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement}"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118004807_1cf50ab7-fac6-432f-baea-daaf5ad2be4a): INSERT INTO TABLE dataset SELECT title, author_id, bestsellers_rank, imprint, cast(to_date(from_unixtime(unix_timestamp(publication_date, 'dd/MM/yyyy'))) as date) as publication_date, rating_avg, rating_count FROM dataset_inter
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:title, type:string, comment:null), FieldSchema(name:author_id, type:int, comment:null), FieldSchema(name:bestsellers_rank, type:int, comment:null), FieldSchema(name:imprint, type:string, comment:null), FieldSchema(name:publication_date, type:date, comment:null), FieldSchema(name:rating_avg, type:float, comment

Comprobamos que los datos están cargados

In [42]:
sql_statement = "SELECT * FROM dataset LIMIT 5;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement}"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118004831_10655656-ee2b-4fa3-9ed4-f6b4845bfb6d): SELECT * FROM dataset LIMIT 5
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:dataset.title, type:string, comment:null), FieldSchema(name:dataset.author_id, type:int, comment:null), FieldSchema(name:dataset.bestsellers_rank, type:int, comment:null), FieldSchema(name:dataset.imprint, type:string, comment:null), FieldSchema(name:dataset.publication_date, type:date, comment:null), FieldSchema(name:dataset.rating_avg, type:float, comment:null), FieldSchema(name:dataset.rating_count, type:int, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20211118004

Borramos la tabla intermedia

In [43]:
sql_statement = "DROP TABLE dataset_inter;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement}"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118004837_71e215de-de79-4dbc-9a77-8ef6c8f87c4c): DROP TABLE dataset_inter
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20211118004837_71e215de-de79-4dbc-9a77-8ef6c8f87c4c); Time taken: 0.047 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118004837_71e215de-de79-4dbc-9a77-8ef6c8f87c4c): DROP TABLE dataset_inter
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20211118004837_71e215de-de79-4dbc-9a77-8ef6c8f87c4c); Time taken: 0.191 seconds
INFO  : OK
INFO 

Comprobamos que están creadas solamente las tablas author y dataset

In [44]:
!beeline -u "jdbc:hive2://localhost:10000/practica" -e "SHOW TABLES;"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118004842_5157b842-aa4b-4ca0-a9a7-23947237c192): SHOW TABLES
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:tab_name, type:string, comment:from deserializer)], properties:null)
INFO  : Completed compiling command(queryId=root_20211118004842_5157b842-aa4b-4ca0-a9a7-23947237c192); Time taken: 0.04 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118004842_5157b842-aa4b-4ca0-a9a7-23947237c192): SHOW TABLES
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20211118004842_5157b842-aa4b-4ca0-a9a7-23947237c192); Time 

### Ejercicio 2.3. Crea una vista sobre las tablas creadas

Esta vista tendrá para cada título, el nombre del autor, fecha de publicación, y valoración media.

In [45]:
sql_statement = "DROP VIEW dataset_view;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement};"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118004847_1f976787-73fc-4965-89db-4b8fead5fcc7): DROP VIEW dataset_view
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20211118004847_1f976787-73fc-4965-89db-4b8fead5fcc7); Time taken: 0.036 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118004847_1f976787-73fc-4965-89db-4b8fead5fcc7): DROP VIEW dataset_view
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20211118004847_1f976787-73fc-4965-89db-4b8fead5fcc7); Time taken: 0.01 seconds
INFO  : OK
INFO  : Co

In [46]:
sql_statement = "CREATE VIEW dataset_view as \
SELECT title, author_name, publication_date, rating_avg \
from author INNER JOIN dataset on author.author_id=dataset.author_id;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement}"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118004852_c6009a4b-b200-42d8-865d-c4e3227f65bd): CREATE VIEW dataset_view as SELECT title, author_name, publication_date, rating_avg from author INNER JOIN dataset on author.author_id=dataset.author_id
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:title, type:string, comment:null), FieldSchema(name:author_name, type:string, comment:null), FieldSchema(name:publication_date, type:date, comment:null), FieldSchema(name:rating_avg, type:float, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20211118004852_c6009a4b-b200-42d8-865d-c4e3227f65bd); Time taken: 0.228 seconds
INFO  : Concurrency mode is d

Comprobar que la vista se ha creado correctamente

In [47]:
! beeline -u "jdbc:hive2://localhost:10000/practica" -e "show tables;"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118004858_6c9bb930-a667-4968-b55a-c771157edd30): show tables
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:tab_name, type:string, comment:from deserializer)], properties:null)
INFO  : Completed compiling command(queryId=root_20211118004858_6c9bb930-a667-4968-b55a-c771157edd30); Time taken: 0.039 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118004858_6c9bb930-a667-4968-b55a-c771157edd30): show tables
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20211118004858_6c9bb930-a667-4968-b55a-c771157edd30); Time

In [48]:
sql_statement = "SELECT * FROM dataset_view LIMIT 5;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement}"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118004902_e813bcb7-1442-4245-98e4-35154fcb8fa3): SELECT * FROM dataset_view LIMIT 5
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:dataset_view.title, type:string, comment:null), FieldSchema(name:dataset_view.author_name, type:string, comment:null), FieldSchema(name:dataset_view.publication_date, type:date, comment:null), FieldSchema(name:dataset_view.rating_avg, type:float, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20211118004902_e813bcb7-1442-4245-98e4-35154fcb8fa3); Time taken: 0.304 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(que

### Ejercicio 2.4. Crea las consultas de Hive necesarias para responder las siguientes cuestiones

#### ¿Cuál es el título del libro con mejor valoración media del autor Rand McNally?

In [49]:
sql_statement = "SELECT author.author_name, dataset.title, dataset.rating_avg FROM author \
INNER JOIN dataset \
ON author.author_id = dataset.author_id and author.author_name = 'Rand McNally' \
order by dataset.rating_avg DESC LIMIT 1;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement}"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118004930_d79e56ef-88f9-4068-863b-90700461b642): SELECT author.author_name, dataset.title, dataset.rating_avg FROM author INNER JOIN dataset ON author.author_id = dataset.author_id and author.author_name = 'Rand McNally' order by dataset.rating_avg DESC LIMIT 1
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:author.author_name, type:string, comment:null), FieldSchema(name:dataset.title, type:string, comment:null), FieldSchema(name:dataset.rating_avg, type:float, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20211118004930_d79e56ef-88f9-4068-863b-90700461b642); Time taken: 0.251 seconds
INFO  :

#### ¿Cuáles son los cinco autores cuyos libros tienen más valoraciones?

In [50]:
sql_statement = " SELECT author.author_name, sum(rating_count) as rating_sum FROM author \
INNER JOIN dataset \
ON author.author_id = dataset.author_id \
GROUP BY author.author_name \
ORDER BY rating_sum DESC LIMIT 5;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement}"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118005106_393cf066-f5cf-4cd7-85e6-02b920a202b5): SELECT author.author_name, sum(rating_count) as rating_sum FROM author INNER JOIN dataset ON author.author_id = dataset.author_id GROUP BY author.author_name ORDER BY rating_sum DESC LIMIT 5
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:author.author_name, type:string, comment:null), FieldSchema(name:rating_sum, type:bigint, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20211118005106_393cf066-f5cf-4cd7-85e6-02b920a202b5); Time taken: 0.226 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(quer

#### ¿Cuáles son los dos libros más recientes del autor Rand McNally?

In [51]:
sql_statement = " SELECT title, publication_date FROM author \
INNER JOIN dataset \
ON author.author_id = dataset.author_id AND author.author_name = 'Rand McNally' \
ORDER BY publication_date DESC \
LIMIT 2;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement}"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118005156_663ea8e6-1311-417f-8b73-55ca68d752d7): SELECT title, publication_date FROM author INNER JOIN dataset ON author.author_id = dataset.author_id AND author.author_name = 'Rand McNally' ORDER BY publication_date DESC LIMIT 2
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:title, type:string, comment:null), FieldSchema(name:publication_date, type:date, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20211118005156_663ea8e6-1311-417f-8b73-55ca68d752d7); Time taken: 0.221 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_2021111800

### Ejercicio 2.5. Selecciona un dataset, impórtalo en las tablas de Hive necesarias e implementa al menos dos consultas sobre dicho dataset.

Para esta parte del ejercicio práctico se ha escogido el dataset FIFA World Cup que contiene información sobre la Copa Mundial de Fútbol desde la primera edición del año 1930 hasta la edición del 2014. Los ficheros que se pretenden cargar almacenan los datos de las ediciones de la Copa Mundial, de todos los partidos disputados y de los jugadores participantes. 

Las preguntas que se desean contestar están relacionadas con la estadística y con la cultura general que forma parte del torneo. 

El dataset se encuentra en el enlace https://www.kaggle.com/abecklas/fifa-world-cup, donde se puede encontrar más información sobre él. 

### Primera parte. Creación de tablas

In [52]:
sql_statement = "DROP TABLE worldcups;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement};"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118005256_0f6fd6b6-c67c-4141-b37b-f68953f6d51b): DROP TABLE worldcups
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20211118005256_0f6fd6b6-c67c-4141-b37b-f68953f6d51b); Time taken: 0.036 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118005256_0f6fd6b6-c67c-4141-b37b-f68953f6d51b): DROP TABLE worldcups
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20211118005256_0f6fd6b6-c67c-4141-b37b-f68953f6d51b); Time taken: 0.009 seconds
INFO  : OK
INFO  : Concu

In [53]:
sql_statement = "CREATE TABLE IF NOT EXISTS worldcups(year INT, country STRING, winner STRING, runners_up STRING, \
thrid STRING, fourth STRING, goalsscored INT, qualifiedteams INT, matchesplayed INT, attendance INT) \
COMMENT 'Copas del mundo con su informacion' \
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' \
TBLPROPERTIES ('Autor' = 'Bigdata', 'Fecha' = '09/11/21', 'skip.header.line.count'='1');"

!beeline -u "jdbc:hive2://localhost:10000/" -e "USE practica; {sql_statement}"

Connecting to jdbc:hive2://localhost:10000/
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118005306_5168024b-7fd7-4d9c-ad8f-7255699b3298): USE practica
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20211118005306_5168024b-7fd7-4d9c-ad8f-7255699b3298); Time taken: 0.053 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118005306_5168024b-7fd7-4d9c-ad8f-7255699b3298): USE practica
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20211118005306_5168024b-7fd7-4d9c-ad8f-7255699b3298); Time taken: 0.008 seconds
INFO  : OK
INFO  : Concurrency mode is disabled,

In [54]:
sql_statement = "DROP TABLE matches;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement};"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118005312_4672a981-e5e0-43a2-b68f-e6429c621862): DROP TABLE matches
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20211118005312_4672a981-e5e0-43a2-b68f-e6429c621862); Time taken: 0.032 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118005312_4672a981-e5e0-43a2-b68f-e6429c621862): DROP TABLE matches
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20211118005312_4672a981-e5e0-43a2-b68f-e6429c621862); Time taken: 0.008 seconds
INFO  : OK
INFO  : Concurren

In [55]:
sql_statement = "CREATE TABLE IF NOT EXISTS matches \
(year INT, datetime STRING, stage STRING, stadium STRING, city STRING, home_team STRING, home_goals INT, \
away_goals INT, away_team STRING, win_conditions STRING, attendance INT, home_goals_ht INT, away_goals_ht INT, \
referee STRING, assitant_one STRING, assistant_two STRING, round_id INT, match_id INT, \
home_team_initials STRING, away_team_initials STRING) \
COMMENT 'Partidos de la copa del mundo con su informacion' \
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' \
TBLPROPERTIES ('Autor' = 'Bigdata', 'Fecha' = '09/11/21', 'skip.header.line.count'='1');"

!beeline -u "jdbc:hive2://localhost:10000/" -e "USE practica; {sql_statement}"

Connecting to jdbc:hive2://localhost:10000/
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118005316_ee1763ff-f25e-45e3-88f8-b37ba1f4f1db): USE practica
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20211118005316_ee1763ff-f25e-45e3-88f8-b37ba1f4f1db); Time taken: 0.042 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118005316_ee1763ff-f25e-45e3-88f8-b37ba1f4f1db): USE practica
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20211118005316_ee1763ff-f25e-45e3-88f8-b37ba1f4f1db); Time taken: 0.01 seconds
INFO  : OK
INFO  : Concurrency mode is disabled, 

In [56]:
sql_statement = "DROP TABLE players;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement};"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118005323_b4f80df2-57a4-4ab0-9cfb-5357c6769772): DROP TABLE players
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20211118005323_b4f80df2-57a4-4ab0-9cfb-5357c6769772); Time taken: 0.031 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118005323_b4f80df2-57a4-4ab0-9cfb-5357c6769772): DROP TABLE players
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20211118005323_b4f80df2-57a4-4ab0-9cfb-5357c6769772); Time taken: 0.008 seconds
INFO  : OK
INFO  : Concurren

In [57]:
sql_statement = "CREATE TABLE IF NOT EXISTS players \
(round_id INT, match_id INT, team_initials STRING, coach STRING, line_up VARCHAR(1), \
shirt_number TINYINT, player STRING, position VARCHAR(2), event STRING) \
COMMENT 'Jugadores de la copa del mundo con su informacion' \
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' \
TBLPROPERTIES ('Autor' = 'Bigdata', 'Fecha' = '09/11/21', 'skip.header.line.count'='1');"

!beeline -u "jdbc:hive2://localhost:10000/" -e "USE practica; {sql_statement}"

Connecting to jdbc:hive2://localhost:10000/
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118005327_ac29cd55-5dc9-4697-bcec-ef64cd5f5946): USE practica
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20211118005327_ac29cd55-5dc9-4697-bcec-ef64cd5f5946); Time taken: 0.029 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118005327_ac29cd55-5dc9-4697-bcec-ef64cd5f5946): USE practica
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20211118005327_ac29cd55-5dc9-4697-bcec-ef64cd5f5946); Time taken: 0.01 seconds
INFO  : OK
INFO  : Concurrency mode is disabled, 

Comprobamos que las tablas se han creado correctamente

In [58]:
sql_statement = "SHOW TABLES;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement}"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118005332_ac289303-d2cf-4009-aa0f-7c06ff035cb2): SHOW TABLES
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:tab_name, type:string, comment:from deserializer)], properties:null)
INFO  : Completed compiling command(queryId=root_20211118005332_ac289303-d2cf-4009-aa0f-7c06ff035cb2); Time taken: 0.036 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118005332_ac289303-d2cf-4009-aa0f-7c06ff035cb2): SHOW TABLES
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20211118005332_ac289303-d2cf-4009-aa0f-7c06ff035cb2); Time

Copiamos los archivos csv a Hadoop

In [59]:
!hadoop fs -copyFromLocal WorldCups.csv /user/root/
!hadoop fs -copyFromLocal WorldCupMatches.csv /user/root/
!hadoop fs -copyFromLocal WorldCupPlayers.csv /user/root/

Cargamos los datos desde los archivos csv a las tablas creadas anteriormente

In [60]:
sql_statement = "LOAD DATA INPATH 'WorldCups.csv' INTO TABLE worldcups;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement};"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118005345_249a1369-53a7-48a2-a954-2c9b5aefa3a3): LOAD DATA INPATH 'WorldCups.csv' INTO TABLE worldcups
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20211118005345_249a1369-53a7-48a2-a954-2c9b5aefa3a3); Time taken: 0.057 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118005345_249a1369-53a7-48a2-a954-2c9b5aefa3a3): LOAD DATA INPATH 'WorldCups.csv' INTO TABLE worldcups
INFO  : Starting task [Stage-0:MOVE] in serial mode
INFO  : Loading data to table practica.worldcups from hdfs://namenode:8020/user/root/W

In [61]:
sql_statement = "LOAD DATA INPATH 'WorldCupMatches.csv' INTO TABLE matches;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement};"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118005349_066097c9-8225-4b73-9928-bb156aa47ce4): LOAD DATA INPATH 'WorldCupMatches.csv' INTO TABLE matches
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20211118005349_066097c9-8225-4b73-9928-bb156aa47ce4); Time taken: 0.058 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118005349_066097c9-8225-4b73-9928-bb156aa47ce4): LOAD DATA INPATH 'WorldCupMatches.csv' INTO TABLE matches
INFO  : Starting task [Stage-0:MOVE] in serial mode
INFO  : Loading data to table practica.matches from hdfs://namenode:8020/user/

In [62]:
sql_statement = "LOAD DATA INPATH 'WorldCupPlayers.csv' INTO TABLE players;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement};"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118005354_69a8d693-ebe2-41a1-9f16-669f68bdc27c): LOAD DATA INPATH 'WorldCupPlayers.csv' INTO TABLE players
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20211118005354_69a8d693-ebe2-41a1-9f16-669f68bdc27c); Time taken: 0.043 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20211118005354_69a8d693-ebe2-41a1-9f16-669f68bdc27c): LOAD DATA INPATH 'WorldCupPlayers.csv' INTO TABLE players
INFO  : Starting task [Stage-0:MOVE] in serial mode
INFO  : Loading data to table practica.players from hdfs://namenode:8020/user/

Comprobamos que los datos se han cargado correctamente

In [63]:
sql_statement = "SELECT * FROM worldcups LIMIT 5;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement}"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118005359_d897addb-00ca-498c-996a-bba27999475a): SELECT * FROM worldcups LIMIT 5
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:worldcups.year, type:int, comment:null), FieldSchema(name:worldcups.country, type:string, comment:null), FieldSchema(name:worldcups.winner, type:string, comment:null), FieldSchema(name:worldcups.runners_up, type:string, comment:null), FieldSchema(name:worldcups.thrid, type:string, comment:null), FieldSchema(name:worldcups.fourth, type:string, comment:null), FieldSchema(name:worldcups.goalsscored, type:int, comment:null), FieldSchema(name:worldcups.qualifiedteams, type:int, comment:null), FieldSchema(na

In [64]:
sql_statement = "SELECT * FROM matches LIMIT 5;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement}"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118005406_7190bf1a-f94f-4944-84dd-68a1ac210613): SELECT * FROM matches LIMIT 5
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:matches.year, type:int, comment:null), FieldSchema(name:matches.datetime, type:string, comment:null), FieldSchema(name:matches.stage, type:string, comment:null), FieldSchema(name:matches.stadium, type:string, comment:null), FieldSchema(name:matches.city, type:string, comment:null), FieldSchema(name:matches.home_team, type:string, comment:null), FieldSchema(name:matches.home_goals, type:int, comment:null), FieldSchema(name:matches.away_goals, type:int, comment:null), FieldSchema(name:matches.away_team, ty

In [65]:
sql_statement = "SELECT * FROM players LIMIT 5;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement}"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118005413_0bc1c604-6765-4a19-a822-660ca154b24f): SELECT * FROM players LIMIT 5
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:players.round_id, type:int, comment:null), FieldSchema(name:players.match_id, type:int, comment:null), FieldSchema(name:players.team_initials, type:string, comment:null), FieldSchema(name:players.coach, type:string, comment:null), FieldSchema(name:players.line_up, type:varchar(1), comment:null), FieldSchema(name:players.shirt_number, type:tinyint, comment:null), FieldSchema(name:players.player, type:string, comment:null), FieldSchema(name:players.position, type:varchar(2), comment:null), FieldSchema(name

### Segunda parte. Creación de consultas

### Países que más veces han perdido una final de la Copa del Mundo (1930-2014) sin haberla ganado nunca, ordenados de mayor a menor subcampeonatos.

Primero tenemos que obtener la lista de países que aparecen como subcampeones (runners_up) y que no aparecen como ganadores (winner). El resultado de la consulta lo guardamos en una tabla temporal "not_winners". 
Una vez que tenemos la lista de los no ganadores hacemos un INNER JOIN con la propia tabla, utilizando como clave de unión el nombre del país. Seguidamente, agrupamos los resultados por el nombre del país para poder realizar una operación COUNT que calcula cuántas veces cada país ha sido subcampeón. Por último, ordenamos la lista en base a este criterio de mayor a menor. 

In [66]:
sql_statement = "SELECT runners_up, count(*) as count \
FROM worldcups INNER JOIN \
(SELECT DISTINCT runners_up FROM worldcups a \
WHERE runners_up NOT IN (SELECT DISTINCT winner FROM worldcups)) AS not_winners \
ON worldcups.runners_up = not_winners.runners_up \
GROUP BY worldcups.runners_up \
ORDER BY count DESC;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement}"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118005437_fa454192-d3df-4eab-b174-dd70155b4e71): SELECT runners_up, count(*) as count FROM worldcups INNER JOIN (SELECT DISTINCT runners_up FROM worldcups a WHERE runners_up NOT IN (SELECT DISTINCT winner FROM worldcups)) AS not_winners ON worldcups.runners_up = not_winners.runners_up GROUP BY worldcups.runners_up ORDER BY count DESC
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Warning: Map Join MAPJOIN[82][bigTable=?] in task 'Stage-13:MAPRED' is a cross product
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:runners_up, type:string, comment:null), FieldSchema(name:count, type:bigint, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=roo

### Número de goles por partidos jugados en cada Copa del Mundo (1930-2014) en orden cronológico.

Hay que realizar una división entre el número de goles marcados, goalsscored, y el número de partidos jugados, matchesplayed. Como el resultado de esta división posiblemente sea un número decimal lo convertimos utilizando la función CAST. Establecemos 4 decimales para obtener un resultado más preciso.

In [69]:
sql_statement = "SELECT country, year, qualifiedteams, goalsscored, matchesplayed, \
cast(goalsscored/matchesplayed as decimal(6, 4)) as goals_matches \
FROM worldcups \
ORDER BY year;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement}"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118005925_99ef002b-70c6-466b-a211-b7035ab90926): SELECT country, year, qualifiedteams, goalsscored, matchesplayed, cast(goalsscored/matchesplayed as decimal(6, 4)) as goals_matches FROM worldcups ORDER BY year
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:country, type:string, comment:null), FieldSchema(name:year, type:int, comment:null), FieldSchema(name:qualifiedteams, type:int, comment:null), FieldSchema(name:goalsscored, type:int, comment:null), FieldSchema(name:matchesplayed, type:int, comment:null), FieldSchema(name:goals_matches, type:decimal(6,4), comment:null)], properties:null)
INFO  : Completed compiling command(que

### Cinco jugadores que más partidos ha ganado jugando, es decir, siendo titulares o saliendo desde el banquillo. 

Primero, se obtienen los jugadores que han sido titulares, line_up toma el valor 'S', o han jugado después de comenzar el partido, en el campo event se busca el caracter 'I'. Seguidamente, se obtienen los equipos ganadores de todos los partidos jugados, es decir, aquellos partidos donde el número de goles del equipo local es distinto del número de goles del equipo visitante. Para recuperar el equipo ganador se utiliza la claúsula IF para comparar los goles de ambos equipos. Tras esto, se realiza un INNER JOIN de ambas consultas agrupando los datos por el id del partido y por las iniciales del equipo de los jugadores. De esta forma se evita que se asocien partidos donde los jugadores no han participado. Por último, se realiza una agrupación de los datos a través del nombre del jugador, se ordenan de mayor a menor según el número de veces que aparecen y se limitan los resultados.

In [70]:
sql_statement = "SELECT player, count(*) as times FROM \
(SELECT distinct player, players_a.match_id, team_initials FROM \
(SELECT match_id, player, team_initials \
FROM players \
WHERE line_up = 'S' OR instr(event, 'I') = 1) as players_a \
INNER JOIN \
(SELECT match_id, IF(home_goals > away_goals, home_team_initials, away_team_initials) as winner \
FROM matches \
WHERE home_goals != away_goals) as match_winners \
ON players_a.match_id = match_winners.match_id AND players_a.team_initials = match_winners.winner) as player_dist \
GROUP BY player \
ORDER BY times DESC \
LIMIT 5;"

!beeline -u "jdbc:hive2://localhost:10000/practica" -e "{sql_statement}"

Connecting to jdbc:hive2://localhost:10000/practica
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20211118010017_568bc588-1680-433b-badd-5e14b06342f1): SELECT player, count(*) as times FROM (SELECT distinct player, players_a.match_id, team_initials FROM (SELECT match_id, player, team_initials FROM players WHERE line_up = 'S' OR instr(event, 'I') = 1) as players_a INNER JOIN (SELECT match_id, IF(home_goals > away_goals, home_team_initials, away_team_initials) as winner FROM matches WHERE home_goals != away_goals) as match_winners ON players_a.match_id = match_winners.match_id AND players_a.team_initials = match_winners.winner) as player_dist GROUP BY player ORDER BY times DESC LIMIT 5
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(nam

# Conclusiones

La realización de esta primera práctica me ha resultado muy interesante y me ha ayudado a comprende los conceptos del primer módulo de la asignatura. He podido entender cómo se utiliza el paradigma MapReduce a la hora de tratar con datos y sus principales ventajas: distribución de trabajo, tolerancia a fallos y mejora de la capacidad global de cómputo. Uno de los aspectos que me hubieran gustado probar es la capacidad de cómputo ante la escalabilidad de los datos, ya que el dataset del que se parte no tiene un número grande de datos. Quizá sería interesante futuras prácticas utilizar un dataset con un número de datos mayor para poner a prueba la capacidad de Hadoop. 

En cuanto a la segunda parte de la práctica, se debe comentar que he decidido realizar la parte de Hive porque tengo conocimientos previos de SQL y he podido comprobar que tienen muchas características en común. También comentar que he valorado positivamente que nos diera a elegir el dataset en esta parte de la práctica. Creo que es una buena forma de motivarnos y de que pensemos en aplicar estos nuevos conocimientos en las áreas que más nos interesan.

Como aspecto a mejorar, he echado en falta algún apartado donde se trabajaran conceptos de inyección de datos, ya que por mi experiencia laboral conozco que es una práctica que se realiza habitualmente. 
Por último, comentar que me han sido de mucha utilidad los ejemplos subidos en el aula virtual así como los ejemplos que se encuentran en el repositorio de GitHub del profesorado.

En definitiva, considero que la práctica ayuda a afianzar los conocimientos del primer módulo, que su realización ha sido más interesante al poder darnos la posibilidad de elejir un conjunto de datos para que trabajemos con él, pero me hubiera gustado poner en práctica más conceptos de la parte de inyección de datos.  